In [1]:
import sys
import traceback
from datetime import datetime

In [2]:
from aiohttp import web
from aiohttp.web import Request, Response, json_response
from botbuilder.core import(BotFrameworkAdapterSettings, TurnContext, BotFrameworkAdapter)
from botbuilder.core.integration import aiohttp_error_middleware
from botbuilder.schema import Activity, ActivityTypes
from bot import MyBot
from config import DefaultConfig

In [3]:
CONFIG = DefaultConfig()

In [4]:
# create adapter 
# https://aka.ms/about-bot-adapter to learn more about how bots work

In [4]:
SETTINGS = BotFrameworkAdapterSettings(CONFIG.APP_ID, CONFIG.APP_PASSWORD)
ADAPTER = BotFrameworkAdapter(SETTINGS)

In [5]:
async def on_error(context: TurnContext, error: Exception):
    
    # this check writes out errors to console log vs app insights
    # note: in production environment, we should consider logging this to Azure application insights
    
    print(f"\n [on_turn_error] unhandled error: {error}", file=sys.stderr)
    traceback.print_exc()
    
    #send a message to the user
    
    await context.send_activity("The bot encountered an error or bug.")
    await context.send_activity("To continue to run this bot, please fix the bot source code.")
    
    # send a trace activity if we´re talking to the Bot Framework Emulator
    
    if context.activity.channel_id == "emulator":
        
        # create a trace activity that contains the error object
        
        trace_activity = Activity(
            
            label = "TurnError",
            
            name = "on_turn_error Trace",
            
            timestamp = datetime.utcnow(),
            
            type = ActivityType.trace,
            
            value = f"{error}",
            
            value_type = "https://www.botframework.com/schemas/error",
        
            )
        await context.send_activity(trace_activity)


In [6]:
ADAPTER.on_turn_error = on_error

In [7]:
# create the Bot

BOT = MyBot()

In [8]:
# Listen for incoming requests on /api/messages

async def messages(req: Request) -> Response:
    
    # Main bot message handler
    
    if "application/json" in req.headers["Content-Type"]:
        
        body = await req.json()
    
    else:
        
        return Response(status=415)
    
    activity = Activity().deserialize(body)
    auth_header = req.headers["Authorization"] if "Authorization" in req.headers else ""
    
    try:
        response = await ADAPTER.process_activity(activity, auth_header, BOT.on_turn)
        
        if response:
            
            return json_response(data=response.body, status=response.status)
        
        return Response(status=201)
    
    except Exception as exception:
        
        raise exception

In [9]:
def init_func(argv):
    
    APP = web.Application(middlewares=[aiohttp_error_middleware])
    
    APP.router.add_post("/api/messages", messages)
    
    return APP

In [10]:
if __name__ == "__main__":
    
    APP = init_func(None)
    
    try:
        
        web.run_app(APP, host="localhost", port=CONFIG.PORT)
    
    except Exception as error:
        
        raise error

RuntimeError: Cannot run the event loop while another loop is running